# Using 3rd Party Library
https://catalog.us-east-1.prod.workshops.aws/workshops/ee59d21b-4cb8-4b3d-a629-24537cf37bb5/en-US/lab2/etl-code

In [1]:
%%bash
git clone https://github.com/jefftune/pycountry-convert.git
cd pycountry-convert
pip install ./

Defaulting to user installation because normal site-packages is not writeable
Processing /home/glue_user/workspace/jupyter_workspace/pycountry-convert
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pycountry-convert: filename=pycountry_convert-0.2.2-py3-none-any.whl size=12451 sha256=f991d36cf3f9d32bdb0d7331c071019fb58188f972c51421c0c96b539e27c1c9
  Stored in directory: /home/glue_user/.cache/pip/wheels/16/26/7a/f1e19b4992a30de1c8f148147a05343f2084d2274f30a280ff
  Created wheel for pycountry: filename=pycountr

Cloning into 'pycountry-convert'...
--- Logging error ---
Traceback (most recent call last):
  File "/home/glue_user/.local/lib/python3.7/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/glue_user/.local/lib/python3.7/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/glue_user/.local/lib/python3.7/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/glue_user/.local/lib/python3.7/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/glue_user/.local/lib/python3.7/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/home/glue_user/.local/lib/python3.7/site-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable =

In [2]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, StringType
from pyspark import SparkContext
from pyspark.sql import SQLContext

from datetime import datetime
from pycountry_convert import (
    convert_country_alpha2_to_country_name,
    convert_country_alpha2_to_continent,
    convert_country_name_to_country_alpha2,
    convert_country_alpha3_to_country_alpha2,
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.load("s3://bucket001/input/lab2/sample.csv", 
                          format="csv", 
                          sep=",", 
                          inferSchema="true",
                          header="true")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def get_country_code2(country_name):
    country_code2 = 'US'
    try:
        country_code2 = convert_country_name_to_country_alpha2(country_name)
    except KeyError:
        country_code2 = ''
    return country_code2

udf_get_country_code2 = udf(lambda z: get_country_code2(z), StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
new_df = df.withColumn('country_code_2', udf_get_country_code2(col("Country")))
new_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- uuid: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- itemtype: string (nullable = true)
 |-- saleschannel: string (nullable = true)
 |-- orderpriority: string (nullable = true)
 |-- orderdate: string (nullable = true)
 |-- region: string (nullable = true)
 |-- shipdate: string (nullable = true)
 |-- unitssold: integer (nullable = true)
 |-- unitprice: double (nullable = true)
 |-- unitcost: double (nullable = true)
 |-- totalrevenue: double (nullable = true)
 |-- totalcost: double (nullable = true)
 |-- totalprofit: double (nullable = true)
 |-- country_code_2: string (nullable = true)

In [6]:
new_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+------------+-------------+---------+--------------------+--------+---------+---------+--------+------------+----------+-----------+--------------+
|     uuid|             country|     itemtype|saleschannel|orderpriority|orderdate|              region|shipdate|unitssold|unitprice|unitcost|totalrevenue| totalcost|totalprofit|country_code_2|
+---------+--------------------+-------------+------------+-------------+---------+--------------------+--------+---------+---------+--------+------------+----------+-----------+--------------+
|535113847|          Azerbaijan|       Snacks|      Online|            C|  10/8/14|Middle East and N...|10/23/14|      934|   152.58|   97.44|   142509.72|  91008.96|   51500.76|            AZ|
|874708545|              Panama|    Cosmetics|     Offline|            L|  2/22/15|Central America a...| 2/27/15|     4551|    437.2|  263.33|   1989697.2|1198414.83|  791282.37|            PA|
|854349935|Sao Tome and Prin..